<a href="https://colab.research.google.com/github/0xrocky/master-thesis/blob/main/neural-model-testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Generic imports
import numpy as np
from google.colab import drive
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import scipy.stats as stats
from bokeh import *

# Neural network imports
import tensorflow as tf
from tensorflow.keras.utils import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import *

In [ ]:
# Load traces from the GDrive folder
LOCAL_DRIVE = '/content/drive'
drive.mount( LOCAL_DRIVE, force_remount=True )
# CYCLES_FILE = '/content/drive/MyDrive/ECCML/RealTraces/B0-2021-11-19-16.54-cycles.npy'
# TRACES_FILE = '/content/drive/MyDrive/ECCML/RealTraces/B0-2021-11-19-16.54-traces.npy'
# CYCLES_FILE = '/content/drive/MyDrive/ECCML/RealTraces/B1-2021-11-24-12.55-cycles.npy'
# TRACES_FILE = '/content/drive/MyDrive/ECCML/RealTraces/B1-2021-11-24-12.55-traces.npy'
# CYCLES_FILE = '/content/drive/MyDrive/ECCML/RealTraces/B2-2021-11-24-13.03-cycles.npy'
# TRACES_FILE = '/content/drive/MyDrive/ECCML/RealTraces/B2-2021-11-24-13.03-traces.npy'
# CYCLES_FILE = '/content/drive/MyDrive/ECCML/RealTraces/B3-2021-11-24-13.08-cycles.npy'
# TRACES_FILE = '/content/drive/MyDrive/ECCML/RealTraces/B3-2021-11-24-13.08-traces.npy'
CYCLES_FILE = '/content/drive/MyDrive/ECCML/RealTraces/B4-2021-11-24-13.13-cycles.npy'
TRACES_FILE = '/content/drive/MyDrive/ECCML/RealTraces/B4-2021-11-24-13.13-traces.npy'
# CYCLES_FILE = '/content/drive/MyDrive/ECCML/RealTraces/B5-2021-11-22-16.44-cycles.npy'
# TRACES_FILE = '/content/drive/MyDrive/ECCML/RealTraces/B5-2021-11-22-16.44-traces.npy'

traces = np.load( TRACES_FILE, allow_pickle=True )
cycles = np.load( CYCLES_FILE, allow_pickle=True )
traces = np.multiply( traces, 175 )

Mounted at /content/drive


In [ ]:
# Macro
REAL_TRACES_NUMBER = 1 # number of traces
ID_TIME = 153  # min time duration of a single sub

In [ ]:
# Per mia comodità costruisco delle funzioni per ottenere un sample, un tag, i cicli di clock
def get_trace_sample( i ):
  return traces[ 0 ][ i ]

def get_cycles_tag( i ):
  return cycles[ 0 ][ i ][ 0 ]

def get_cycles_clock( i ):
  return cycles[ 0 ][ i ][ 1 ]

In [ ]:
# Per tutti i tag nell'array, filtro solo sui tag dell'ADD e della MUL
trace_samples_tags = []
# Costruisco il vettore di rilevazioni di ID e MUL, appendendo ad ogni sample di ciascuna operazione un tag booleano
def appending_bool_flag_at_samples( trace_samples_tags, operation_time, clock_cicles_op_curr, bool_tag ):
  for j in range( operation_time ):
    trace_samples_tags += [ ( get_trace_sample( clock_cicles_op_curr + j ), bool_tag ) ]

for h in range( REAL_TRACES_NUMBER ):
  for i in range( len( cycles[ 0 ] ) - 1 ):
    tag_op_curr = get_cycles_tag( i )     # il tag dell'operazione corrente
    clock_cicles_op_curr = get_cycles_clock( i )   # i cicli di clock da cui parte l'operazione corrente
    operation_time = get_cycles_clock( i + 1 ) - clock_cicles_op_curr # calcolo la durata di un'operazione come la differenza tra i cicli di clock registrati all'inizio dell'operazione successiva e quelli dell'operazione corrente - 1
    if clock_cicles_op_curr + operation_time >= len( traces[ 0 ] ):  # il numero di samples è 10mln, ma il numero di cicli di clock registrati è maggiore
      break
    if tag_op_curr == 4:                  # sottrazione
      appending_bool_flag_at_samples( trace_samples_tags, operation_time, clock_cicles_op_curr, True )
    else:                                 # moltiplicazione
      appending_bool_flag_at_samples( trace_samples_tags, operation_time, clock_cicles_op_curr, False )

In [ ]:
print( "Dimensione del vettore delle tracce:", len( trace_samples_tags ) )
cycles = []
traces = []
temp = 0
for i in range( len( trace_samples_tags ) ):
  if trace_samples_tags[ i ][ 1 ] == True:
    temp += 1
print( "Numero di SUB True samples:", temp )
temp = 0
for i in range( len( trace_samples_tags ) ):
  if trace_samples_tags[ i ][ 1 ] == False:
    temp += 1
print( "Numero di False samples:", temp )

Dimensione del vettore delle tracce: 7027186
Numero di SUB True samples: 524247
Numero di False samples: 6502939


In [ ]:
trace_samples = []
trace_tags = []

for single_trace in range( REAL_TRACES_NUMBER ):
  temp_trace_sample = []
  temp_trace_tag = []
  for smpl_tag in range( len( trace_samples_tags ) ):
    temp_trace_sample += [ trace_samples_tags[ smpl_tag ][ 0 ] ]
    temp_trace_tag += [ trace_samples_tags[ smpl_tag ][ 1 ] ]
  trace_samples += [ temp_trace_sample ]
  trace_tags += [ temp_trace_tag ]

size_trace_samples = len( trace_samples )
size_trace_tags = len( trace_tags )
size_single_trace_samples = len( trace_samples[ 0 ] )
size_single_trace_tags = len( trace_tags[ 0 ] )

print( "Size of the list of all real traces (numbers of executions):", REAL_TRACES_NUMBER )
print( "Size of the list of all traces samples constructed from the real traces:", size_trace_samples )
print( "Size of the list of all traces tags constructed from the real traces:", size_trace_tags )
print( "Size of the only first samples trace", size_single_trace_samples )
print( "Size of the only first tags trace", size_single_trace_tags )

if REAL_TRACES_NUMBER != size_trace_samples or REAL_TRACES_NUMBER != size_trace_tags:
  print( "ERROR: number of total samples not equal at number of total tags in the real traces" )
if size_single_trace_samples != size_single_trace_tags:
  print( "ERROR: number of samples of the first trace not equal at number of tags of the first trace" )

Size of the list of all real traces (numbers of executions): 1
Size of the list of all traces samples constructed from the real traces: 1
Size of the list of all traces tags constructed from the real traces: 1
Size of the only first samples trace 7027186
Size of the only first tags trace 7027186


In [ ]:
# Sliding window to create the dataset: fixing a window size and an advancement step, the aim is focusing on an Hamming weight window and tag
# with a 1 the window contains a TRUE tag, 0 otherwise
window = 500
step = 10
X_full_dataset = []
Y_full_dataset = []

for trace in range( REAL_TRACES_NUMBER ):
  for i in range( 0, size_single_trace_samples - window + 1, step ):
    X_full_dataset += [ trace_samples[ trace ][ i:i + window ] ]
    window_of_tags = np.array( trace_tags[ trace ][ i:i + window ] )
    set_True = 0
    # It detects where chunks begin, has some logic for the first and last chunk, and simply computes differences between chunk starts and discards lengths corresponding to False chunks
    arr = np.diff( np.where( np.concatenate( ( [ window_of_tags[ 0 ] ], window_of_tags[ :-1 ] != window_of_tags[ 1: ], [ True ] ) ) )[ 0 ] )[ ::2 ]
    if np.size( arr ) > 0 and np.max( arr ) >= ID_TIME:
      set_True = 1
    Y_full_dataset += [ set_True ]

size_X_full_dataset = len( X_full_dataset )
size_Y_full_dataset = len( Y_full_dataset )
size_single_trace_X = len( X_full_dataset[ 0 ] )

#print( X_full_dataset[ :10 ] )
#print( Y_full_dataset[ :1000 ] )
print( "Size of the X dataset of {} sliding window of samples:".format( window ), size_X_full_dataset )
print( "Size of the Y dataset of {} sliding window of tags:".format( window ), size_Y_full_dataset )
print( "Size of the first trace of the X dataset of samples in sliding window:", size_single_trace_X )
trace_samples = []
trace_tags = []
trace_samples_tags = []

Size of the X dataset of 500 sliding window of samples: 702669
Size of the Y dataset of 500 sliding window of tags: 702669
Size of the first trace of the X dataset of samples in sliding window: 500


In [ ]:
yes = 0 # numero di finestre taggate a 1 nel GT
no = 0 # numero di finestre taggate a 0 nel GT
true = 0 # numero di true di fila
groundtruth = []

for i in range( size_Y_full_dataset ):
  if Y_full_dataset[ i ] == 1:  # Finestra corrente a 1
    yes += 1
    true += 1
  else:                         # Finestra corrente a 0
    no += 1
    if true > 0:
      if true >= 15:
        if true >= 40:
          print( "True di seguito:", true, " - numero finestra: ", i, " - SUB lunga: overflow!" )
          groundtruth += [ ( 1, i ) ]   # SUB lunga
        else:
          print( "True di seguito:", true, " - numero finestra: ", i, " - SUB corta: no overflow" )
          groundtruth += [ ( 0, i ) ]
      else:
        print( "True di seguito: ", true, " - numero finestra: ", i, " - falso positivo" )
      true = 0
print( "Finestre taggate a 1:", yes )
print( "Finestre taggate a 0:", no )

True di seguito: 15  - numero finestra:  15  - SUB corta: no overflow
True di seguito: 59  - numero finestra:  906  - SUB lunga: overflow!
True di seguito: 69  - numero finestra:  1536  - SUB lunga: overflow!
True di seguito: 44  - numero finestra:  1842  - SUB lunga: overflow!
True di seguito: 51  - numero finestra:  2140  - SUB lunga: overflow!
True di seguito: 45  - numero finestra:  2431  - SUB lunga: overflow!
True di seguito: 45  - numero finestra:  2481  - SUB lunga: overflow!
True di seguito: 44  - numero finestra:  3330  - SUB lunga: overflow!
True di seguito: 85  - numero finestra:  3665  - SUB lunga: overflow!
True di seguito: 44  - numero finestra:  3956  - SUB lunga: overflow!
True di seguito: 44  - numero finestra:  4249  - SUB lunga: overflow!
True di seguito: 36  - numero finestra:  4300  - SUB corta: no overflow
True di seguito: 69  - numero finestra:  5188  - SUB lunga: overflow!
True di seguito: 51  - numero finestra:  5793  - SUB lunga: overflow!
True di seguito: 36

In [ ]:
# Load model
print("Loading model...")
model = load_model("/content/drive/My Drive/ECCML/RealTraces/Models/real_model_LSTM1000_AVGPOOL_allOP_notshuffle.model")
print("Done.")

Loading model...
Done.


In [ ]:
## Testing
X_full_dataset = np.array( X_full_dataset )
X_full_dataset = np.reshape( X_full_dataset, ( X_full_dataset.shape[ 0 ], X_full_dataset.shape[ 1 ], 1 ) )
Y_full_dataset = to_categorical( np.array( Y_full_dataset ) )

print( "Shapes:" )
print( "\tX_test: " + str( X_full_dataset.shape ) )
print( "\tY_test: " + str( Y_full_dataset.shape ) )

print( "Computing predictions..." )
predictions = model.predict( X_full_dataset )

print( "Evaluating model..." )
model.evaluate( X_full_dataset, Y_full_dataset )

print( "Done." )

Shapes:
	X_test: (702669, 500, 1)
	Y_test: (702669, 2)
Computing predictions...
Evaluating model...
21959/21959 [==============================] - 752s 34ms/step - loss: 0.0702 - accuracy: 0.9728
Done.


The following code produces an output of this form: `<Network guess> -> <Real value> [<Right guess?>]`.

In [ ]:
win = 500
iter = 10
start = 0
end = 5000
for p in range( len( predictions[ start:end ] ) ):
  print( "Windows n°", p+start, " --- clocks: [", (p+start)*iter, ",", win + (p+start)*iter, "]", str( np.argmax( predictions[ p ] ) ) + " -> " + str( np.argmax( Y_full_dataset[ p ] ) ) + " [" + str( np.argmax( predictions[ p ] ) == np.argmax( Y_full_dataset[ p ] ) ) + "] \t -> " + str( np.round( predictions[ p ], 4 ) ) )

Windows n° 0  --- clocks: [ 0 , 500 ] 1 -> 1 [True] 	 -> [0.0055 0.9945]
Windows n° 1  --- clocks: [ 10 , 510 ] 1 -> 1 [True] 	 -> [0.0058 0.9942]
Windows n° 2  --- clocks: [ 20 , 520 ] 1 -> 1 [True] 	 -> [0.0066 0.9934]
Windows n° 3  --- clocks: [ 30 , 530 ] 1 -> 1 [True] 	 -> [0.0066 0.9934]
Windows n° 4  --- clocks: [ 40 , 540 ] 1 -> 1 [True] 	 -> [0.006 0.994]
Windows n° 5  --- clocks: [ 50 , 550 ] 1 -> 1 [True] 	 -> [0.0066 0.9934]
Windows n° 6  --- clocks: [ 60 , 560 ] 1 -> 1 [True] 	 -> [0.0023 0.9977]
Windows n° 7  --- clocks: [ 70 , 570 ] 1 -> 1 [True] 	 -> [0.005 0.995]
Windows n° 8  --- clocks: [ 80 , 580 ] 1 -> 1 [True] 	 -> [0.0061 0.9939]
Windows n° 9  --- clocks: [ 90 , 590 ] 1 -> 1 [True] 	 -> [0.004 0.996]
Windows n° 10  --- clocks: [ 100 , 600 ] 1 -> 1 [True] 	 -> [0.0071 0.9929]
Windows n° 11  --- clocks: [ 110 , 610 ] 1 -> 1 [True] 	 -> [0.0083 0.9917]
Windows n° 12  --- clocks: [ 120 , 620 ] 1 -> 1 [True] 	 -> [0.0156 0.9844]
Windows n° 13  --- clocks: [ 130 , 630 

In [ ]:
fake = 0
guessed = 0
for p in range( len( predictions ) ):
  if np.argmax( predictions[ p ] ) == np.argmax( Y_full_dataset[ p ] ):
    guessed += 1
  else:
    fake += 1
print( "Quante finestre la rete ha indovinato:" + str( guessed ) )
print( "Quante finestre la rete ha sbagliato:" + str( fake ) )

Quante finestre la rete ha indovinato:683536
Quante finestre la rete ha sbagliato:19133


In [ ]:
sub_overflow_predict = []

for p in range( len( predictions ) ):
  if np.argmax( predictions[ p ] ) == 1:
    true += 1
  else:
    if true > 0:
      if true >= 15:
        if true >= 40:
          print( "True di seguito:", true, "numero predizione: ", p, "SUB lunga: overflow" )
          sub_overflow_predict += [ ( 1, p ) ]
        else:
          print( "True di seguito:", true, "numero predizione: ", p, "SUB corta: no overflow" )
          sub_overflow_predict += [ ( 0, p ) ]
      else:
        print( "True di seguito: ", true, "numero predizione: ", p )
      true = 0

True di seguito: 19 numero predizione:  19 SUB corta: no overflow
True di seguito: 89 numero predizione:  911 SUB lunga: overflow
True di seguito: 28 numero predizione:  1195 SUB corta: no overflow
True di seguito: 70 numero predizione:  1537 SUB lunga: overflow
True di seguito: 42 numero predizione:  1842 SUB lunga: overflow
True di seguito: 39 numero predizione:  2128 SUB corta: no overflow
True di seguito: 91 numero predizione:  2481 SUB lunga: overflow
True di seguito: 45 numero predizione:  3330 SUB lunga: overflow
True di seguito: 88 numero predizione:  3668 SUB lunga: overflow
True di seguito: 46 numero predizione:  3959 SUB lunga: overflow
True di seguito: 44 numero predizione:  4249 SUB lunga: overflow
True di seguito: 34 numero predizione:  4299 SUB corta: no overflow
True di seguito:  1 numero predizione:  4726
True di seguito:  12 numero predizione:  5108
True di seguito: 77 numero predizione:  5186 SUB lunga: overflow
True di seguito: 22 numero predizione:  5489 SUB corta:

In [ ]:
print( len( groundtruth ) )
groundtruth = np.array( groundtruth )
print( "Groundtruth 1:", np.count_nonzero( groundtruth == 1, axis=0 ) )
print( "Groundtruth 0:", np.count_nonzero( groundtruth == 0, axis=0 ) )
print( len( sub_overflow_predict ) )
sub_overflow_predict = np.array( sub_overflow_predict )
print( "Prediction SUB overflow 1:", np.count_nonzero( sub_overflow_predict == 1, axis=0 ) )
print( "Prediction SUB overflow 0:", np.count_nonzero( sub_overflow_predict == 0, axis=0 ) )

1770
Groundtruth 1: [1214    0]
Groundtruth 0: [556   0]
1794
Prediction SUB overflow 1: [1440    0]
Prediction SUB overflow 0: [354   0]
